<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Image Processing Laboratory Notebooks</h2>
<hr style="clear:both">
<p style="font-size:0.85em; margin:2px; text-align:justify">
This Juypter notebook is part of a series of computer laboratories which are designed
to teach image-processing programming; they are running on the EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course <b>Image Processing I</b> 
(<a href="https://moodle.epfl.ch/course/view.php?id=522">MICRO-511</a>) taught by Prof. M. Unser and Prof. D. Van de Ville.
</p>
<p style="font-size:0.85em; margin:2px; text-align:justify">
The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the <a href="http://bigwww.epfl.ch/">Biomedical Imaging Group</a>. 
The distribution or the reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL <mark>2021</mark>.
</p>
<p style="font-size:0.85em; margin:0px"><b>Authors</b>: 
    <a href="mailto:pol.delaguilapla@epfl.ch">Pol del Aguila Pla</a>, 
    <a href="mailto:kay.lachler@epfl.ch">Kay Lächler</a>,
    <a href="mailto:alejandro.nogueronaramburu@epfl.ch">Alejandro Noguerón Arámburu</a>, and
    <a href="mailto:daniel.sage@epfl.ch">Daniel Sage</a>.
</p>
<hr style="clear:both">
<h1>Lab 2.2: Filtering Applications</h1>
<div style="background-color:#F0F0F0;padding:4px">
    <p style="margin:4px;"><b>Released</b>: <mark>Thursday November 10, 2022</mark></p>
    <p style="margin:4px;"><b>Submission</b>: <mark><span style="color:red">Friday November 18, 2022</span></mark> (before 11:59PM) on <a href="https://moodle.epfl.ch/course/view.php?id=522">Moodle</a></p>
    <p style="margin:4px;"><b>Grade weight</b>: Lab 2 (<mark>16</mark> points), 10% of the overall grade</p>
    <p style="margin:4px;"><b>Help sessions</b>: <mark>Thursday November 17</mark> on campus</p>        
    <p style="margin:4px;"><b>Related lectures</b>: Chapter 3</p>
</div>

### Student Name: 
### SCIPER: 

Double-click on this cell and fill your name and SCIPER number. Then, run the cell below to verify your identity in Noto and set the seed for random results.

In [ ]:
%use sos
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

### <a name="imports_"></a> Imports
In the next cell we import the Python libraries that we will use throughout the lab, as well as the `ImageViewer` class (Python package developed specifically for these laboratories, see documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), or run the python command `help(viewer)` after loading the class):

* [`matplotlib.pyplot`](https://matplotlib.org), to display images
* [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/), to make the image display interactive
* [`numpy`](https://numpy.org/doc/stable/reference/index.html), for mathematical operations on arrays
* [`openCV` (cv2)](https://docs.opencv.org/2.4/index.html), for image-processing tasks
* [`scipy.ndimage`](https://docs.scipy.org/doc/scipy/reference/ndimage.html), Scipy's specific module for multidimensional image processing
* [`scikit-image` (skimage)](https://scikit-image.org/docs/stable/api/api.html), also for image-processing tasks

We will then load the `ImageViewer` class (either see the complete documentation [here](https://github.com/Biomedical-Imaging-Group/interactive-kit/wiki/Image-Viewer), run the Python command `help(viewer)` after loading the class, or refer to [Lab 0: Introduction](../0_Introductory_lab/Introductory.ipynb#4.-Python-Image-Viewer)).

Finally, we load the images you will use in the exercise to test your algorithms. 

In [ ]:
%use sos
# Configure plotting as dynamic
%matplotlib widget

# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv 
import scipy.ndimage as ndi
import ipywidgets as widgets
import skimage
import scipy
import scipy.stats
from skimage import filters
from interactive_kit import imviewer as viewer

# Load images to be used in this exercise 
bikesgray = cv.imread('images/bikesgray.tif', cv.IMREAD_UNCHANGED).astype('float64')

# Import the whole stack of 1200 images from camera-16bits.tif
from skimage import io
camera = io.imread("images/camera-16bits.tif").astype(np.float64)
# # We extract one of the images as a test image
camera_test = camera[55]

Now we will import the `ImageAccess` class. You can find the documentation of the class [here](https://biomedical-imaging-group.github.io/image-access/). Moreover, we will put the images we will be using in the JavaScript kernel.

In [ ]:
%use javascript
%get bikesgray camera_test
// import IPLabImageAccess as Image
var Image = require('image-access')

# Filtering applications (7 points)

After the [first part](./1_Filtering.ipynb) of the lab, we expect you to feel comfortable with the basics of filtering. In this part we will look in the detail at the implementation of a Gaussian filter, as well as some of its direct applications. Gaussian filters are known to be near-optimal smoothing filters, and represent perhaps the most used preprocessing step in image processing to improve robustness in a workflow and to denoise images.

## <a id="ToC_2_FilteringApplications"></a>Table of contents
1. [Gaussian filter](#1.-Gaussian-filter-(4-points))
    1. [Implementation of a 2D Gaussian filter](#1.A.-Implementation-of-a-2D-Gaussian-filter-(4-points)) (**4 points**)
    2. [Gaussian filter in Python](#1.B.-Gaussian-filter-in-Python)
4. [Application: Super-resolution localization microscopy](#2.-Application:-Super-resolution-localization-microscopy-(3-points))
    1. [Difference of Gaussians](#2.A.-Difference-of-Gaussians-(1-points)) (**1 points**)
    2. [Local maxima](#2.B.-Local-maxima-(1-point)) (**1 point**)
    3. [Center of gravity](#2.C.-Center-of-gravity-(1-point)) (**1 point**)
    4. [Complete localization pipeline](#2.D.-Complete-localization-pipeline)

<div class=" alert alert-danger">

<b>Important:</b> Each cell that contains code begins with `%use sos` or `%use javascript`. This indicates if the code in this specific cell should be written in Python or JavaScript. Do not change or remove any lines of code that begin with an %. They are used for the notebook to run smoothly with <code>SoS</code> and need to be on the first line of each cell!
</div>

Good luck and enjoy! 

### Visualize images
Get familiar now with the images you are going to be using.

Remember that to use the `ImageViewer` class, we only need to call it with an image (and make sure that the image is a `numpy.ndarray`, or a list of such arrays). From there you can change the plotting range, visualize the histogram, get the statistics, browse through the images with the buttons `Next` & `Prev`, etc.

In [ ]:
%use sos
# Declare image_list for ImageViewer
image_list = [bikesgray, camera_test]

imgs_viewer = viewer(image_list, widgets=True, hist=True)

# 1. Gaussian filter (4 points)
[Back to table of contents](#ToC_2_FilteringApplications)

In this section you will implement a 2D Gaussian filter with impulse response $h_{\sigma}[m,n]$, where $\sigma$ is the standard deviation of an isotropic 2D Gaussian and controls the smoothing strength. This impulse response discretizes the 2D Gaussian function $h_\sigma(x,y)$ between $[-\lceil3\sigma\rceil,\lceil 3\sigma\rceil]$ in $x$ and $y$. You will **choose the size of the filter to be $N = 2\lceil 3\sigma \rceil+1$** (hence, $N$ is always odd), and you will **ensure your impulse response adds up to $1$** using the appropriate normalization. Here, $\lceil x \rceil$ refers to the ceiling function, i.e., the smallest integer larger than a given $x\in\mathbb{R}$. 

Remember that the expression of an [isotropic 2D Gaussian](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) is

$$h_\sigma(x,y) = \frac{1}{2\pi\sigma^2}\exp\left(-\frac{x^2 + y^2}{2\sigma^2}\right) = \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{x^2}{2\sigma^2}\right) \frac{1}{\sqrt{2\pi}\sigma}\exp\left(-\frac{ y^2}{2\sigma^2}\right) \,.$$

## 1.A. Implementation of a 2D Gaussian filter (4 points)
[Back to table of contents](#ToC_2_FilteringApplications)

For **3 points**, implement the function `gaussian(img, sigma)` that convolves an image with a Gaussian filter using a **separable implementation**. If you wish, you can take advantage of the `filter1D` function defined in Section [1.B.](./1_Filtering.ipynb#1.B.-Separable-implementation-(2-points)) and redefined here.

<div class="alert alert-success">
    <b>Hint: </b> Remember that you can use <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Math">the <code>Math</code> library</a> to access different mathematical functions (<code>Math.ceil()</code>, <code>Math.floor()</code>, <code>Math.exp()</code>, <code>Math.PI</code>, <code>Math.sqrt()</code>).
</div>

<div class="alert alert-info">

<b>Remember:</b> The first argument to the <code>Image</code> constructor is the <b>height</b> and the second is the <b>width</b>: <code>new Image(height, width)</code> or <code>new Image([height, width])</code>. Feel free to go back to <a href="../0_Introductory_lab/Introductory.ipynb#3.-Javascript-image-access-class-(2-points)">Lab 0: Introduction</a> to review the basic usage of the ImageAccess class.
</div>

In [ ]:
%use javascript
// function that performs a gaussian filter with sigma on img
function gaussian(img, sigma){
    // declare output variable
    var output = new Image(img.shape());
    
    // Define normalized 1D mask
    // YOUR CODE HERE
    
    // Filter using separable implementation (hint: your mask should be 1D)
    // You can use the filter1D function defined below
    // YOUR CODE HERE
    
    return output
}

// function that applies a 1D filter
function filter1D(img, mask){
    // transpose the input variables if necessary
    if(img.nx == 1){
        img.transposeImage();
    }
    if(mask.nx == 1){
        mask.transposeImage();
    }
    // create the output image
    var output = new Image(img.shape());
    // iterate through all pixels
    for(var x = 0; x < img.nx; x++){
        // get the neighbourhood around position x
        var neigh = img.getNbh(x, 0, mask.nx, 1);
        // declare a variable to store the values of the convolution. 
        var val = 0;
        // iterate through the neighbourhood
        for(var i = 0; i < neigh.nx; i++){
            // perform convolution
            val += neigh.getPixel(i, 0) * mask.getPixel(mask.nx - 1 - i, 0);
        }
        // set value in output array
        output.setPixel(x, 0, val);
    }
    return output
}

We have designed a quick test for you to evaluate your method, applying it to a $3 \times 3$ impulse image. Run the following cell and check that your output has all the desired properties of a Gaussian.

In [ ]:
%use javascript
// define the impulse image
var impulse = new Image([[0, 0, 0], [0, 1, 0], [0, 0, 0]]);

// apply filter to previously defined impulse
var impulse_gaussian = gaussian(impulse, 0.5);

// look at result, verify that it has the properties of a Gaussian
console.log('Your impulse Gaussian:\n' + impulse_gaussian.visualize());

// this assertion checks proper behaviour: that the center is the maximum, and that two pixels in equivalent positions have the same values.
if(impulse_gaussian.getPixel(1, 1) < impulse_gaussian.getPixel(1, 2) || impulse_gaussian.getPixel(1, 0) !== impulse_gaussian.getPixel(1, 2)){
    console.log('WARNING!!!\nThere are still some mistakes with your implementation! Look at the sanity checks to understand the mistakes');
}else{
    console.log('The symmetry of the Gaussian seems good.');
}

// check normalization
var sum = 0
for(var x = 0; x < impulse_gaussian.nx; x++){
    for (var y = 0; y < impulse_gaussian.ny; y++){
        sum += impulse_gaussian.getPixel(x, y);
    }
}
if(Math.abs(sum - 1) > 1e-5){
    console.log("WARNING!!\nNormalization not correct");
}else{
    console.log("Well done! The output sums up to approximately 1.");
}

Now that you have tested your Gaussian filter, apply it to the image `bikesgray`. Use different values of $\sigma$ (you can change it in the next cell). Look at the evolution of the mean and the standard deviation (you can get them from the statistics box in the `viewer`, or you can use the functions `np.mean` and `np.std`). Then, answer the two multiple choice questions.

Run and modify the two following cells to apply Gaussian filters with different $\sigma$ values to `bikesgray` and view the result. 

In [ ]:
%use javascript
%put bikesgray_gaussian1 bikesgray_gaussian5

// apply filter to Image object. To try different sigma values, change the variables or declare more. 
var bikesgray_gaussian1 = gaussian(new Image(bikesgray), 1).toArray()
var bikesgray_gaussian5 = gaussian(new Image(bikesgray), 5).toArray()

In [ ]:
%use sos

# Declare parameters for ImageViewer. If you want to visualize more sigma values, update the previous cell and these lists accordingly
image_list_blur = [bikesgray, bikesgray_gaussian1, bikesgray_gaussian5]
title_list_blur = ['Original', 'Sigma: 1', 'Sigma: 5']
# Make sure that the object used is a numpy array
for i in range(len(image_list_blur)):
    image_list_blur[i] = np.array(image_list_blur[i])

# To allow a direct comparison of the images.
plt.close('all')
blurred_bikesgray_viewer = viewer(image_list_blur, title=title_list_blur, hist=True)

### Multiple Choice Question

After modifying the two cells above and visualizing the results, answer the next two questions (worth **0.5 points** each).

* Q1: How would you expect the Fourier transform to change?
    1. It will show lower values for higher frequencies.
    2. It will show higher values for higher frequencies.
    3. It will show lower values for lower frequencies.
    4. It will not change.


* Q2: What will be the output image when $\sigma\rightarrow \infty$? What type of filter would that be?
    1. An image equal to the original. It would be an all-pass filter.
    2. A constant image. It would be a high-pass filter.
    3. A 2D Gaussian. It would be a band-pass filter.
    4. A constant image. It would be a low-pass filter.

Modify the variables `answer_one` and `answer_two` in the next cell to match your choices. The second and third cells are for you to make sure that your answer is in the valid range (they should not raise any error).

In [ ]:
%use sos
# Modify these variables
answer_one = None
answer_two = None
# YOUR CODE HERE

In [ ]:
%use sos
# Sanity test
if not answer_one in [1, 2, 3, 4, 5]:
    print('WARNING!\nAnswer one of 1, 2, 3, 4 or 5.')

In [ ]:
%use sos
# Sanity test
if not answer_two in [1, 2, 3, 4]:
    print('WARNING!\nAnswer one of 1, 2, 3 or 4.')

## 1.B. Gaussian filter in Python
[Back to table of contents](#ToC_2_FilteringApplications)

There are several implementations of Gaussian filters in Python. In this section, we will use the `scikit-image` implementation (see its documentation [here](https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian)). The basic syntax is the following:

```python
output = skimage.filters.gaussian(input, sigma, mode, truncate, preserve_range = True)
```

The parameters are:
* `input` (numpy array): Original image.
* `sigma` (float): $\sigma$ value.
* `mode` (string): Boundary conditions. As in the rest of the course, we will be using `'reflect'`. Look carefully at its description in [this link](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.ndimage.convolve.html#scipy.ndimage.convolve).
* `truncate` (float): truncate the filter at this many standard deviations. Defaults to 4, but **we will use 3** to agree with the implementation in JavaScript.
* `preserve_range` (boolean): indicates whether to convert the image to a floating point value between $0$ and $1$ (`False`), or simply normalizing the Gaussian kernel to sum to one (`True`). 

The output is the filtered image. 

<div class = 'alert alert-info'>
<b>Note</b>: In this lab we have presented -and will use- <i>Skimages'</i> implementation of the Gaussian filter because it is more adequate to what we need. However, we encourage you to check out OpenCV's and SciPy's implementation!
</div>

In the next cells, we will give you an example on the image `bikesgray`. Furthermore, we will compare it to your implementation. Run the next cell to get a blurred version of bikesgray using the skimage gaussian filter. We will use $\sigma = 10$.

In [ ]:
%use sos
# Apply a gaussian filter with sigma=10 to the bikesgray image
bikesgray_gaussian_skimage = skimage.filters.gaussian(bikesgray, sigma=10 , mode='reflect', truncate=3, preserve_range=True)
gaussian_viewer = viewer(bikesgray_gaussian_skimage)

Now, we will compare it to your implementation in JavaScript to make sure that they are equivalent (up to errors on the order of $10^{-14}$). For this, we call the `gaussian` function you implemented with the image bikesgray, also for $\sigma = 10$. 

Run the next cell to get the variable `bikesgray_gaussian10`, and the one below it to make the comparison in Python. Note that, if the images are not the same, an `ImageViewer` will pop up, showing you in red the regions that differ the most. If necessary, you can use this information to try to find your mistakes. 

In [ ]:
%use javascript
%put bikesgray_gaussian10

// apply filter to Image object
var bikesgray_gaussian10 = gaussian(new Image(bikesgray), 10).toArray()

Now we will look at each of them and at their differences. Look at the range of values in the histogram to verify the scale of the differences.

In [ ]:
%use sos

# Make sure that the one imported from JavaScript is a numpy array
bikesgray_gaussian10 = np.array(bikesgray_gaussian10)

# Declare parameters of viewer
image_list = [bikesgray_gaussian10, bikesgray_gaussian_skimage, np.abs(bikesgray_gaussian_skimage - bikesgray_gaussian10)]
title_list = ['JS', 'Skimage', 'Difference']

# We call the viewer with clip_range = [0, 1] to compare the difference with respect to the originals
plt.close('all')
if not np.allclose(bikesgray_gaussian10, bikesgray_gaussian_skimage):
    print('The results of your Gaussian filter do not match Skimage results! Look at the red areas in the viewer to see where you might have gone wrong.')
    skimage_gaussian_viewer = viewer([bikesgray_gaussian10, bikesgray_gaussian_skimage], title=['JS', 'Skimage'], widgets=True, compare=True)
else :
    print('Seems like your Gaussian filter is correct!')

# 2. Application: Super-resolution localization microscopy (3 points)
[Back to table of contents](#ToC_2_FilteringApplications)

Now that you have implemented several filters and you master the concepts behind digital filtering, we are going to lead you in a real application of Gaussian filtering. You are going to implement a *Basic Localization Tool for Super-resolution Localization Microscopy*, based on the [Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians) (DoG) filter. From now on, you will only be using Python libraries.

In this exercise, we introduce a basic pipeline (see image below) to analyse SMLM images (single-molecule localization microscopy). SMLM is a light microscopy technique which allows to break the famous [Abbe diffraction limit](https://en.wikipedia.org/wiki/Diffraction-limited_system#The_Abbe_diffraction_limit_for_a_microscope) (resolution ~250 nm). SMLM achieves super-resolution performances (resolution ~25 nm) by localising blinking fluorescent molecules (emitters) isolated in different frames (images). To obtain a good reconstruction, SMLM requires a very long sequence (10'000 to 100'000 frames) at very low density of emitters in a frame, 2 to 10 bright spots. The highly contrasted spots are easy to detect and it is possible to localize their center at a sub-pixel accuracy in order to reconstruct a super-resolution map.

A good algorithm to detect such spots is to compute the local maximum on the output of the DoG filter ([Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians)), which is an approximation of the Laplacian-of-Gaussian (LoG) filter explained on _page 5-35_ of the course notes.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="Localization showcase" src="images/localization_showcase.jpg" width="900"><br>
      <em style="color: grey">SMLM Pipeline: The pipeline starts with centering the mean of the image at $0$. This is followed by a DoG filter, and the local maxima of the result is obtained. Finally, the center of gravity of each image is obtained, and the information is used to construct the high-resolution final image. See the dedicated sections for more details.</em>
  </p>
</td>
</tr></table>

For this exercise, we will use the image sequence `camera`, which consists of 1200 individual frames of single-molecule emitter spots. First of all, let's look at some example frames from the `camera` image sequence. Run the next cell to look at 4 randomly selected frames (you can rerun the cell multiple times to see different frames).

In [ ]:
%use sos
# Define the zero_mean function, which you already coded in the previous lab, so no need to do it again
def zero_mean(img):
    return img - np.mean(img)

plt.close('all')
rand_idx = np.random.randint(0, camera.shape[0]-1, size=4)
view = viewer([camera[i] for i in rand_idx], title=[f'camera frame {i}' for i in rand_idx], subplots=(2,2))

## 2.A. Difference of Gaussians (1 points)
[Back to table of contents](#ToC_2_FilteringApplications)

The first step in the localization pipeline is to produce the zero-mean version of each frame. The function `zero_mean`, which takes care of that has already been defined for you in the previous cell. The next step is to implement the [Difference of Gaussians](https://en.wikipedia.org/wiki/Difference_of_Gaussians) filter.

The DoG is constructed from the difference between two Gaussian functions, i.e., $\mathrm{DoG}(x) = h_{\sigma_{1}}(x) - h_{\sigma_2}(x)$. It is usually parametrised only by $\sigma_1$, and $\sigma_2$ is chosen as $\sigma_2 = \sqrt{2}\sigma_1$. Experiment with the value of $\sigma_1$ in the next cell to see the kind of profile generated by this filter in 1D (use the slider to do so). 

In [ ]:
%use sos
# Choose sigmas
sigma_slider = widgets.FloatSlider(value=0.5, min=0.5, max=5, description='$\sigma_1$')

# Initialize figure
plt.close('all')
fig, ax = plt.subplots(1, 1, num=f"Difference of Gaussians filter in 1D - SCIPER: {uid}", figsize=[10,4])
ax.plot(0, 0, 0, 0, 0, 0);
ax.set_xlabel(r"$x$"); plt.legend([r"$h_{\sigma_1}(x)$", r"$h_{\sigma_2}(x)$", r"$\mathrm{DoG}_{\sigma_1}(x)$"]);
ax.set_xlim([-20, 20]); ax.set_ylim([-0.1, 0.8])
ax.grid(); fig.tight_layout()
 
# Plotting function - Callback for slider
def dog_1d(change):
    # Get value of sigma, initialize variables of interest and clear axes
    sigma_1 = change.new
    sigma_2 = sigma_1*np.sqrt(2)
    # Update plot
    x = np.arange(-3*sigma_2, (3+6./100)*sigma_2, 6*sigma_2/100)
    ax.lines[0].set_data(x, scipy.stats.norm(scale=sigma_1).pdf(x))
    ax.lines[1].set_data(x, scipy.stats.norm(scale=sigma_2).pdf(x))
    ax.lines[2].set_data(x, scipy.stats.norm(scale=sigma_1).pdf(x) - scipy.stats.norm(scale=sigma_2).pdf(x));

sigma_slider.observe(dog_1d, 'value')
sigma_slider.value = 1
display(sigma_slider)

For **1 point**, modify the next cell and write the function `dog`, that takes as input an image and the value of $\sigma_1$. This function should implement the DoG filter by combining the result of two Gaussian filters. **You should hardcode the value of $\sigma_2 = \sigma_1\sqrt{2}$**. After getting the *DoG*, normalize the output of your function so that it spans the range $[0,1]$.

<div class = 'alert alert-info'>
<b>Note</b>: in Section <a href="#1.B.-Gaussian-filter-in-Python">1.B.</a> we presented you the <code>scikit-image</code> implementation of the Gaussian filter, which is simple and clear ($2$ strong points of SciKit-Image). However, as you will see later, OpenCV and SciPy tend to be faster, and have different strong points. Of course we will accept any correct implementation, but we encourage you to find OpenCV's and SciPy's implementations and use the one you prefer! <b>Make sure you use the correct parameters, like <code>preserve_range=True</code> option (or equivalent), truncating the filter at $3\sigma$ (so $N = 2\lceil 3\sigma \rceil+1$), and using <code>'reflect'</code> or equivalent boundary conditions.</b>
</div>

Complete the function `dog` in the next cell, where we have also included an initial sanity check.

In [ ]:
%use sos

def dog(image, sigma_1):
    output = np.zeros(image.shape)
    
    # Apply the DoG filter to image
    # YOUR CODE HERE
    
    return output

if dog(camera_test, 1).max() != 1 or dog(camera_test, 1).min() != 0:
    print("WARNING: Remember to normalize the output so that it spans the range [0, 1].")

In the next two cells, you will visualize the results of your function for different $\sigma$ values. We will declare a slider with values in the range $[0.2-3]$, a button and an activation function to get the value of the slider and apply your function to an input image. 

First, run the next cell to declare these widgets and the function.

In [ ]:
%use sos
# Define sliders and button
sigma_slider = widgets.FloatSlider(value=1, min=0.2, max=3.0, step=0.1, description='\u03c3\u2081:')
button = widgets.Button(description='Apply DoG')
# Define callback function
def button_dog(image):
    sigma = sigma_slider.value
    image = dog(image, sigma)
    return image

Now run the next cell to visualize the results. Go to the menu *Extra Widgets*, where you can find the slider. You will apply it to the image `camera_test`.

In [ ]:
%use sos
# Visualize the dog camera_test
plt.close("all")
dog_viewer = viewer(camera_test, title="DoG camera_test", new_widgets=[sigma_slider, button], 
                    callbacks=[button_dog], widgets=True, normalize=True)

## 2.B. Local maxima (1 point)
[Back to table of contents](#ToC_2_FilteringApplications)

Now you will write the function `local_max(img, T)` that returns a binary image. This function will set the pixels which are a local maximum in a $3\times 3$ neighbourhood to $255$, and any other pixels to $0$. A local maximum is a pixel that has a value strictly greater than its 8 closest neighbors (8-connected) and is strictly greater than a threshold $T$ (specified between $0$ and $1$, relative to the maximum of the image).

<div class="alert alert-info">
    <b>Remember:</b> 8-connected pixels are neighbors to every pixel that touches one of their edges or corners.<br>
    <table><tr>
    <td>
      <p align="center" style="padding: 0px">
        <img src="images/8_connectivity.jpg" alt="8-connectivity" width="100px"><br>
      </p>
    </td>
    </tr></table>
</div>

<div class = 'alert alert-success'>
<b>Hint</b>: Remember that Image Processing libraries can do most of the work for you. <code>scikit-image</code> has the method <code>skimage.feature.peak_local_max</code> (<a href = 'https://scikit-image.org/docs/0.7.0/api/skimage.feature.peak'>see documentation here</a>), which allows you to specify the size of the neighborhood (through the parameter <code>min_distance</code>, that leads to regions of size <code>2*min_distance+1</code>) and a threshold relative to the maximum value of the image. It outputs the coordinates of the corresponding pixels. Once you have the coordinates, you can use them to index an array and put the desired value in the appropriate places. For example, if we have a NumPy array <code>peaks</code> with local maxima locations as the one returned by the aforementioned function, you can index an image as <code>output[peaks[:,0], peaks[:,1]] = value</code>.
    
We strongly suggest you use this function. If you are curious about the algorithm, remember that in the documentation of most Python libraries, you can find the <a href = 'https://github.com/scikit-image/scikit-image/blob/main/skimage/feature/peak.py#L119-L326'>source code</a> of a function.
</div>
    
For **1 point**, modify the next cell and define your function.

In [ ]:
%use sos
# Import the module feature from skimage if you are going to use it
# Use it as feature.peak_local_max
from skimage import feature

# Function that computes the local max in a 3x3 nbh
def local_max(img, T):
    output = np.zeros(img.shape)
    
    # Apply the local maxima
    # YOUR CODE HERE
    
    return output

Run the next cell for a quick test on your function. In it, we test  that your image applied to `camera_test` with a threshold $T = 0.5$ detects exactly the six maximum points of the image, as it should.

In [ ]:
%use sos
plt.close('all')
view = viewer([camera_test, local_max(camera_test, 0.5)], title=['camera_test', 'local maxima'], subplots=(1,2))
if np.count_nonzero(local_max(camera_test, 0.5)) == 6:
    print('Congratulations! Your function passed this initial sanity check.')
else :
    print('WARNING!!\nYour function is not working on the image `camera_tests` as it should.')

Now you are going to see the effect of this function through a slider on the `ImageViewer`. Like in the exercise to visualize the effects of `dog`, we will declare one slider for the threshold and one button to call the `local_max` method. Run the next cell to test it on the image `camera_test`.

In [ ]:
%use sos

threshold_slider = widgets.FloatSlider(value=0, min=0, max=1, step=0.01, description='T:')
button = widgets.Button(description='Apply Local Maxima')

def button_local_max(image):
    t = threshold_slider.value
    image = local_max(image, t)
    return image

local_max_viewer = viewer(camera_test, title="Local Maxima", new_widgets=[threshold_slider, button], 
                          callbacks=[button_local_max], widgets=True)

## 2.C. Center of gravity (1 point)
[Back to table of contents](#ToC_2_FilteringApplications)

The most important part of the localization pipeline is the calculation of the center of gravity. This is what allows to **increase the resolution by a factor of 16** in our case. The center of gravity $\operatorname{c}$ of a $3 \times 3$ pixel window **centered around the index (0,0)** is calculated as follows:

$$\operatorname{c}(x, y) = \left(\frac{\sum\limits_{x,y=-1}^{x,y=1}x\operatorname{w}(x,y)}{\sum\limits_{x,y=-1}^{x,y=1}\operatorname{w}(x,y)}, \frac{\sum\limits_{x,y=-1}^{x,y=1}y\operatorname{w}(x,y)}{\sum\limits_{x,y=-1}^{x,y=1}\operatorname{w}(x,y)}\right),$$
where $\operatorname{w}(x,y) = \operatorname{f}(x,y) - \operatorname{min}_{x,y=-1}^{x,y=1}(\operatorname{f}(x,y))$
and $\operatorname{f}(x,y)$ is the $3 \times 3$ pixel window.

<table><tr>
<td>
  <p align="center" style="padding: 0px">
    <img alt="Center of gravity" src="images/cog.png" width="200"><br>
  </p>
</td>
</tr></table>

Remember that using `for` loops in python is very inefficient, so try to avoid them whenever possible. One way to get around using `for` loops to implement the sums in this case is to use the function [`np.meshgrid`](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) together with [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html). The way we use [`np.meshgrid`](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) here is to produce the two arrays
$$X=
\begin{bmatrix}
-1 & 0 & 1\\
-1 & 0 & 1\\
-1 & 0 & 1
\end{bmatrix}
\text{ and }
Y=
\begin{bmatrix}
-1 & -1 & -1\\
0 & 0 & 0\\
1 & 1 & 1
\end{bmatrix}.$$

Think about how you can use these two matrices together with [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html) to avoid `for` loops and **for 1 point** implement the function `cog` in the cell below.
<div class='alert alert-info'>
    <b>Hint:</b> The center of gravity should be between $-1$ and $+1$ in both directions.
</div>

In [ ]:
%use sos

# Function that calculates the center of gravity of a 3x3 window
def cog(f):
    cog_x = 0
    cog_y = 0
    
    # Create coordinate system
    X, Y = np.meshgrid([-1, 0, 1], [-1, 0, 1])
    
    # Calculate center of gravity
    # YOUR CODE HERE
    
    return cog_x, cog_y

Run the next cell to perform some simple sanity checks on 4 test windows.

In [ ]:
%use sos
# Create test windows
test_windows = [np.zeros((3, 3)) for i in range(4)]
test_windows[0][0, 0] = 1
test_windows[1][2, 2] = 1
test_windows[2][1, 1] = 1
test_windows[3][1, 1:3] = 1
correct_cogs = [(-1.0, -1.0), (1.0, 1.0), (0.0, 0.0), (0.5, 0.0)]
# Display test windows
plt.close('all')
view = viewer(test_windows, title=[f'test window {i}' for i in range(4)], subplots=(2,2))
# Check output
check = True
for i in range(len(test_windows)):
    print(f'Cog of test window {i}: {cog(test_windows[i])}')
    if not np.allclose(cog(test_windows[i]), correct_cogs[i]):
        print(f'WARNING: The cog of test window {i} should be {correct_cogs[i]}.\n')
        check = False
if check:
    print('\nNice, your cog function passed the sanity checks.')
else :
    print('\nSorry, there are still some errors in your code.')

## 2.D. Complete localization pipeline
[Back to table of contents](#ToC_2_FilteringApplications)

Finally, we'll use the functions you implemented to define the complete localization pipeline. We will also look at the difference between a simple reconstruction without localization and the one that you created now. Run the next cell to define both the `reconstruction` and the `simple_reconstruction` functions, which we already implemented for you.

In [ ]:
def reconstruct(img, sigma_1, T, out_shape=(512, 512)):
    out = np.zeros((img.shape[0], out_shape[0], out_shape[1]))
    # Calculate size ratio between low-res and high-res image
    r_x = out_shape[1] // img.shape[2]
    r_y = out_shape[0] // img.shape[1]
    
    # Apply localization pipeline to each image
    for i in range(img.shape[0]):
        # Zero mean
        img_zm = zero_mean(img[i])
        # DoG
        img_dog = dog(img_zm, sigma_1)
        # Extract coordinates of the local maxima
        coords = np.where(local_max(img_dog, T))
        # Cog
        for i in range(len(coords[0])):
            # Calculate center of gravity
            cog_x, cog_y = cog(img_zm[coords[0][i]-1:coords[0][i]+2, coords[1][i]-1:coords[1][i]+2])
            # Set the corresponding pixel in the high-res image to 255
            out[i, int(np.round(coords[0][i] * r_y + cog_y * (3*r_y/2))), 
                   int(np.round(coords[1][i] * r_x + cog_x * (3*r_x/2)))] = 255
    
    # Combine all images and threshold
    out = np.where(np.sum(out, axis=0) > 0, 255, 0)
    
    return out

def simple_reconstruct(img, sigma_1, T, out_shape=(512, 512)):
    img_lm = np.zeros(img.shape)
    for i in range(img.shape[0]):
        # Detect peaks
        img_norm = zero_mean(img[i])
        img_dog = dog(img_norm, sigma_1)
        img_lm[i] = local_max(img_dog, T)
    
    # Average peaks
    img_avg = np.mean(img_lm, axis=0)
    # Resize to higher resolution using cubic interpolation
    out = cv.resize(img_avg, dsize=out_shape, interpolation=cv.INTER_CUBIC)
    return out

Now, run the cell below to visualize both methods and look at the results. **Feel free to experiment with different values for $\sigma_1$ (DoG) and the threshold $T$ (local max)!** Change the values in the next cell to do so.
<div class='alert alert-info'>
    <b>Note:</b> The viewer is <code>joint_zoom</code> mode, so you can zoom into both images at the same time and compare them in detail.
</div>

In [ ]:
# Set sigma and threshold
sig = 0.8
T = 0.5

# Perform reconstructions
rec_simple = simple_reconstruct(camera, sigma_1=sig, T=T)
rec = reconstruct(camera, sigma_1=sig, T=T)
plt.close('all')
view = viewer([rec_simple, rec], title=['Simple reconstruction', 'Localization reconstruction'], subplots=(1,2), joint_zoom=True)

<div class="alert alert-success">
    
<p><b>Congratulations on finishing Lab 2!</b></p>
<p>
Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to <a href="https://moodle.epfl.ch/mod/assign/view.php?id=1111434">Moodle</a>, in a zip file together with the first part of this lab.
</p>
</div>

* Keep the name of the notebook as: *2_Filtering_Applications.ipynb*,
* Name the zip file: *Filtering_lab.zip*.